In [1]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams.update({"font.size": 12})
%matplotlib inline

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacytextblob.spacytextblob import SpacyTextBlob
import json

In [ ]:
!pip3 install spacymoji==2.0.0
!pip3 uninstall spacy -y
!pip3 install spacy==3

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


    - Are there differences between "old" users (pre-2021) and new ones?
    - Are there clusters of users? 
    - Create corpora and embeddings for users based on text submitted by them, upvotes etc.
    - Can we identify those that have posted (successful) DDs? 
    - Can we rank the users according to their expertise?
    - Are the same users active in the other reviewed subreddits? 

In [3]:
df = pd.read_csv("/content/drive/MyDrive/WSB/WSB_submissions_all.csv", sep=";")
df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s')
df.index = pd.DatetimeIndex(df.created_utc)
df.tail()

,Unnamed: 0,author,created_utc,id,link_flair_text,num_comments,score,subreddit_subscribers,selftext,title,upvote_ratio
created_utc,,,,,,,,,,,
2018-12-31 15:05:18,861747,Leading_Breath,2018-12-31 15:05:18,ab7upy,NaN,417,1,454678,NaN,"""I don't give a shit about Tesla I just want t...",0.0
2018-12-31 14:45:20,861748,skitty2,2018-12-31 14:45:20,ab7oz6,NaN,8,1,454674,The whole next year will be green.,If today closes green,0.0
2018-12-31 14:40:30,861749,Googs84,2018-12-31 14:40:30,ab7nkw,Shitpost,0,1,454672,Happy New Year to all you degenerates includin...,Happy New Year Degenerates,0.0
2018-12-31 14:34:10,861750,neocoff,2018-12-31 14:34:10,ab7lu7,NaN,7,1,454672,NaN,Chip stocks set for worst quarter in a decade....,0.0
2018-12-31 14:29:50,861751,Brodmann,2018-12-31 14:29:50,ab7km1,Discussion,2,1,454671,NaN,Aphria down 7% premarket; Questions Surface if...,0.0


# Split datasets 

In [4]:
df['year'] = pd.DatetimeIndex(df['created_utc']).year

In [5]:
df['month'] = pd.DatetimeIndex(df['created_utc']).month
df['date'] = pd.DatetimeIndex(df['created_utc']).date
df['day'] = pd.DatetimeIndex(df['created_utc']).day
df.head()

,Unnamed: 0,author,created_utc,id,link_flair_text,num_comments,score,subreddit_subscribers,selftext,title,upvote_ratio,year,month,date,day
created_utc,,,,,,,,,,,,,,,
2021-10-19 17:58:57,0,VendiVendiVendi,2021-10-19 17:58:57,qbh10s,Gain,0,1,10967090,So 13.56 cad for BB. It’s moving and gaining ...,BB,1.0,2021,10,2021-10-19,19
2021-10-19 17:57:49,1,Far_Bass_7284,2021-10-19 17:57:49,qbh04w,News,1,1,10967087,NaN,Surely it is a coincidence.,1.0,2021,10,2021-10-19,19
2021-10-19 17:55:24,2,HistoricalAnimator43,2021-10-19 17:55:24,qbgyco,YOLO,0,1,10967076,NaN,yolo sava options,1.0,2021,10,2021-10-19,19
2021-10-19 17:54:51,3,ZaRaWi,2021-10-19 17:54:51,qbgxy8,YOLO,0,1,10967076,NaN,meme,1.0,2021,10,2021-10-19,19
2021-10-19 17:52:33,4,CuteInterest1884,2021-10-19 17:52:33,qbgw9u,DD,0,1,10967075,"BB is on the move higher, not because of the p...",BB,1.0,2021,10,2021-10-19,19


In [6]:
df['link_flair_text'] = df['link_flair_text'].astype("string")

In [8]:
df.head(3)

,Unnamed: 0,author,created_utc,id,link_flair_text,num_comments,score,subreddit_subscribers,selftext,title,upvote_ratio,year,month,date,day
created_utc,,,,,,,,,,,,,,,
2021-10-19 17:58:57,0,VendiVendiVendi,2021-10-19 17:58:57,qbh10s,Gain,0,1,10967090,So 13.56 cad for BB. It’s moving and gaining ...,BB,1.0,2021,10,2021-10-19,19
2021-10-19 17:57:49,1,Far_Bass_7284,2021-10-19 17:57:49,qbh04w,News,1,1,10967087,NaN,Surely it is a coincidence.,1.0,2021,10,2021-10-19,19
2021-10-19 17:55:24,2,HistoricalAnimator43,2021-10-19 17:55:24,qbgyco,YOLO,0,1,10967076,NaN,yolo sava options,1.0,2021,10,2021-10-19,19


## group by user

In [9]:
df_authors = df.groupby("author")["author"].count().sort_values(ascending=False)
print(f"Top 15 authors:\n{df_authors.head(15)}")

Top 15 authors:
author
[deleted]               31133
Onboarding92             1103
WSBVoteBot               1094
AutoModerator             923
OPINION_IS_UNPOPULAR      498
1poundbookingfee          310
WSBConsensus              304
bigbear0083               290
pdwp90                    239
patrickbateman02          225
RealPennyMuncher          223
btoned                    220
rawrtherapy               209
keenfeed                  182
Ned_Flanderz              179
Name: author, dtype: int64


In [ ]:
# remove submissions where author = '[deleted]'

In [10]:
df_deleted = df[df['author'] == '[deleted]']
df = df[df['author'] != '[deleted]']

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 830619 entries, 2021-10-19 17:58:57 to 2018-12-31 14:29:50
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Unnamed: 0             830619 non-null  int64         
 1   author                 830619 non-null  object        
 2   created_utc            830619 non-null  datetime64[ns]
 3   id                     830619 non-null  object        
 4   link_flair_text        784349 non-null  string        
 5   num_comments           830619 non-null  int64         
 6   score                  830619 non-null  int64         
 7   subreddit_subscribers  830619 non-null  int64         
 8   selftext               223761 non-null  object        
 9   title                  830619 non-null  object        
 10  upvote_ratio           830619 non-null  float64       
 11  year                   830619 non-null  int64         
 12  month     

In [12]:
df_deleted.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 31133 entries, 2021-10-19 15:46:35 to 2019-01-07 22:05:14
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Unnamed: 0             31133 non-null  int64         
 1   author                 31133 non-null  object        
 2   created_utc            31133 non-null  datetime64[ns]
 3   id                     31133 non-null  object        
 4   link_flair_text        30277 non-null  string        
 5   num_comments           31133 non-null  int64         
 6   score                  31133 non-null  int64         
 7   subreddit_subscribers  31133 non-null  int64         
 8   selftext               58 non-null     object        
 9   title                  31133 non-null  object        
 10  upvote_ratio           31133 non-null  float64       
 11  year                   31133 non-null  int64         
 12  month                  31

In [14]:
df_authors = df.groupby("author")["author"].count().sort_values(ascending=False)
print(f"Top 15 authors:\n{df_authors.head(20)}")

Top 15 authors:
author
Onboarding92            1103
WSBVoteBot              1094
AutoModerator            923
OPINION_IS_UNPOPULAR     498
1poundbookingfee         310
WSBConsensus             304
bigbear0083              290
pdwp90                   239
patrickbateman02         225
RealPennyMuncher         223
btoned                   220
rawrtherapy              209
keenfeed                 182
Ned_Flanderz             179
swaggymedia              173
indonesian_activist      163
cuki321                  162
dhiral1994               160
VisualMod                156
Janto_2021               150
Name: author, dtype: int64


In [15]:
df_daily_title= df.groupby(['author']).agg({'title': ' '.join})
#df_daily_title= df.groupby(pd.Grouper(freq="D")).agg({'title': ' '.join})
#df.groupby(['name','month'])['text'].apply(','.join).reset_index()

In [19]:
df_author_posts= df.groupby(['author']).agg({'id': 'count'})

In [20]:
df_author_posts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 396126 entries, ------------------f to zzzzzzzzzzzzzzzzs
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   id      396126 non-null  int64
dtypes: int64(1)
memory usage: 16.0+ MB


In [21]:
df_author_posts.head()

,id
author,
------------------f,1
--------Username----,1
--------sam,2
------13------,1
-----1------2,1


In [22]:
df_author_posts['title']= df.groupby(['author']).agg({'title': ' '.join})

In [23]:
df_author_posts.head()

,id,title
author,,
------------------f,1,All my homies hate options. $GEVO 🚀🚀
--------Username----,1,Is berkshire B spiking soon?
--------sam,2,She Definitely Bought TSLA Puts 🐻 Holy shit NB...
------13------,1,Robinhood allowing limited shares to be bought...
-----1------2,1,Is it worth investing more now?


In [24]:
df['selftext'] = df['selftext'].astype("string")

In [25]:
df.selftext = np.where(df.selftext.isnull()," ",df.selftext)

In [26]:
df_author_posts['selftext'] = df.groupby(['author']).agg({'selftext': ' '.join})

In [28]:
df['date'] = df['date'].astype("string")

In [ ]:
df_author_posts['date']= df.groupby(['author']).agg({'date': ' '.join})

In [32]:
df['day'] = df['day'].astype("string")
df['month'] = df['month'].astype("string")
df['year'] = df['year'].astype("string")

In [33]:
df_author_posts['day']= df.groupby(['author']).agg({'day': ' '.join})
df_author_posts['month']= df.groupby(['author']).agg({'month': ' '.join})
df_author_posts['year']= df.groupby(['author']).agg({'year': ' '.join})

In [34]:
df_author_posts.head()

,id,title,selftext,date,day,month,year
author,,,,,,,
------------------f,1,All my homies hate options. $GEVO 🚀🚀,,2020-12-03,3,12,2020
--------Username----,1,Is berkshire B spiking soon?,Im 17 and I gave $700 to my brother to buy sto...,2021-09-17,17,9,2021
--------sam,2,She Definitely Bought TSLA Puts 🐻 Holy shit NB...,,2020-10-02 2020-03-12,2 12,10 3,2020 2020
------13------,1,Robinhood allowing limited shares to be bought...,,2021-01-28,28,1,2021
-----1------2,1,Is it worth investing more now?,"Idk how to word it better, im pretty retarded....",2021-01-25,25,1,2021


## number of users who have posted more than n times

In [35]:
df_author_posts[df_author_posts['id'] > 10].info()

<class 'pandas.core.frame.DataFrame'>
Index: 7253 entries, -A-Brocoma2021 to zxc123zxc123
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7253 non-null   int64 
 1   title     7253 non-null   object
 2   selftext  7253 non-null   object
 3   date      7253 non-null   object
 4   day       7253 non-null   object
 5   month     7253 non-null   object
 6   year      7253 non-null   object
dtypes: int64(1), object(6)
memory usage: 453.3+ KB


In [36]:
df_author_posts[df_author_posts['id'] > 1].info()

<class 'pandas.core.frame.DataFrame'>
Index: 138749 entries, --------sam to zzzzzzzzzzzzazzz
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        138749 non-null  int64 
 1   title     138749 non-null  object
 2   selftext  138749 non-null  object
 3   date      138749 non-null  object
 4   day       138749 non-null  object
 5   month     138749 non-null  object
 6   year      138749 non-null  object
dtypes: int64(1), object(6)
memory usage: 8.5+ MB


In [43]:
df_author_posts[(df_author_posts['id'] > 10) & (df_author_posts['id'] <50)].info()

<class 'pandas.core.frame.DataFrame'>
Index: 6977 entries, -A-Brocoma2021 to zxc123zxc123
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        6977 non-null   int64 
 1   title     6977 non-null   object
 2   selftext  6977 non-null   object
 3   date      6977 non-null   object
 4   day       6977 non-null   object
 5   month     6977 non-null   object
 6   year      6977 non-null   object
dtypes: int64(1), object(6)
memory usage: 436.1+ KB


In [38]:
df_author_posts[df_author_posts['id'] > 50].info()

<class 'pandas.core.frame.DataFrame'>
Index: 266 entries, 0toHeroInvesting to zman1175
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        266 non-null    int64 
 1   title     266 non-null    object
 2   selftext  266 non-null    object
 3   date      266 non-null    object
 4   day       266 non-null    object
 5   month     266 non-null    object
 6   year      266 non-null    object
dtypes: int64(1), object(6)
memory usage: 16.6+ KB


In [37]:
df_author_posts[df_author_posts['id'] > 100].info()

<class 'pandas.core.frame.DataFrame'>
Index: 53 entries, 1poundbookingfee to tmime1
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        53 non-null     int64 
 1   title     53 non-null     object
 2   selftext  53 non-null     object
 3   date      53 non-null     object
 4   day       53 non-null     object
 5   month     53 non-null     object
 6   year      53 non-null     object
dtypes: int64(1), object(6)
memory usage: 3.3+ KB


#Tickers by user

In [44]:

nasdaq_stocks = pd.read_csv("/content/drive/MyDrive/WSB/nasdaq.csv", sep=",")
nasdaq_stocks_symbols = nasdaq_stocks["Symbol"].tolist()

In [45]:
# Function to find stocks in a text given stock list
import re
regex = re.compile("[^a-zA-Z ]")
ticker_set = set(nasdaq_stocks_symbols)
#list(filter(lambda word: word[0]=='s', text.split()))

def calculate_mentioned_stocks(title, body=""):
    #content = regex.sub("", str(title) + " " + str(body)).split(" ")
    text = str(title) + " " + str(body)   
    tick_list = list(filter(lambda word: word[0]=='$', text.split())) 
    content = set(list([i[1:] for i in tick_list]))
    tickers = str(list(ticker_set & content))    
    return pd.Series([tickers])

def sanitize_column(column):
    #print(str(column))
    column = str(column).replace("[],", "").replace("[]", "").replace("],[", ",").replace("][", ",")
    return column

In [53]:
df_author_posts['tickers_'] = df_author_posts.apply(lambda x: calculate_mentioned_stocks(x["title"],x["selftext"]), axis=1)


In [54]:
df_author_posts['tickers_'] = df_author_posts['tickers_'].apply(sanitize_column)

In [57]:
df_author_posts.head()

,id,title,selftext,date,day,month,year,tickers_
author,,,,,,,,
------------------f,1,All my homies hate options. $GEVO 🚀🚀,,2020-12-03,3,12,2020,['GEVO']
--------Username----,1,Is berkshire B spiking soon?,Im 17 and I gave $700 to my brother to buy sto...,2021-09-17,17,9,2021,
--------sam,2,She Definitely Bought TSLA Puts 🐻 Holy shit NB...,,2020-10-02 2020-03-12,2 12,10 3,2020 2020,
------13------,1,Robinhood allowing limited shares to be bought...,,2021-01-28,28,1,2021,
-----1------2,1,Is it worth investing more now?,"Idk how to word it better, im pretty retarded....",2021-01-25,25,1,2021,


In [59]:
df_author_posts['tickers_'].tolist()[:10]

["['GEVO']", '', '', '', '', '', '', '', '', '']

In [60]:
df_author_posts['tickers']= df_author_posts['tickers_'].apply(lambda x: sanitize_column(x))

In [67]:
df_author_posts['tickers'].tolist()[:10]

["['GEVO']", '', '', '', '', '', '', '', '', '']

In [68]:
from collections import Counter, defaultdict
import ast

In [69]:
#count stocks
df_author_posts["tickers"] =df_author_posts["tickers"].str.strip("[]").str.split(",")
df_author_posts["tickers_count"] = df_author_posts.apply(lambda x: dict(Counter(list(x['tickers']))), axis=1)

In [76]:
from numpy.core.numeric import NaN

def agg_counter_objs(rows):
    #print(rows)
    #print(type(rows))
    if ((rows == '') | (rows == NaN)):
        return 0
    new_counter_objs = []
    #rows= ast.literal_eval(rows)
    def default_val():
        return 0
        
    new_dict = defaultdict(default_val)
    #print(rows)
    test_dict = Counter(rows)
    for key in test_dict.keys():
        new_key = key.replace("'","").replace('"',"").replace("]","").replace("\\","").strip()
        new_dict[new_key] += test_dict[key]
            
    if "" in new_dict.keys():
        del new_dict[""]
            
    new_counter_objs.append(Counter(new_dict))
        
    agg_counter_obj = Counter({})
    for obj in new_counter_objs:
        agg_counter_obj += obj        
    return (str(dict(agg_counter_obj)))

In [77]:
df_author_posts['tickers_count_'] = df_author_posts["tickers_count"].apply(agg_counter_objs)

In [81]:
df_author_posts = df_author_posts.drop(['tickers','tickers_count'],axis=1)

In [82]:
df_author_posts.head()

,id,title,selftext,date,day,month,year,tickers_,tickers_count_
author,,,,,,,,,
------------------f,1,All my homies hate options. $GEVO 🚀🚀,,2020-12-03,3,12,2020,['GEVO'],{'GEVO': 1}
--------Username----,1,Is berkshire B spiking soon?,Im 17 and I gave $700 to my brother to buy sto...,2021-09-17,17,9,2021,,{}
--------sam,2,She Definitely Bought TSLA Puts 🐻 Holy shit NB...,,2020-10-02 2020-03-12,2 12,10 3,2020 2020,,{}
------13------,1,Robinhood allowing limited shares to be bought...,,2021-01-28,28,1,2021,,{}
-----1------2,1,Is it worth investing more now?,"Idk how to word it better, im pretty retarded....",2021-01-25,25,1,2021,,{}


In [85]:
df['link_flair_text'] = df['link_flair_text'].astype("string")

In [86]:
df['link_flair_text'] = np.where(df.link_flair_text.isnull(),"",df.link_flair_text)

In [87]:
df_author_posts['link_flair_text']= df.groupby(['author']).agg({'link_flair_text': ' '.join})

In [90]:
df_author_posts['comments_mean']= df.groupby(['author']).agg({'num_comments': 'mean'})
df_author_posts['comments_sum']= df.groupby(['author']).agg({'num_comments': 'sum'})

In [92]:
df_author_posts['score_mean']= df.groupby(['author']).agg({'score': 'mean'})
df_author_posts['score_sum']= df.groupby(['author']).agg({'score': 'sum'})

In [93]:
df_author_posts['upvote_ratio_mean']= df.groupby(['author']).agg({'upvote_ratio': 'mean'})
df_author_posts['upvote_ratio_sum']= df.groupby(['author']).agg({'upvote_ratio': 'sum'})

In [94]:
df_author_posts.head()

,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum
author,,,,,,,,,,,,,,,,
------------------f,1,All my homies hate options. $GEVO 🚀🚀,,2020-12-03,3,12,2020,['GEVO'],{'GEVO': 1},Stocks,0.0,0,1.0,1,1.0,1.0
--------Username----,1,Is berkshire B spiking soon?,Im 17 and I gave $700 to my brother to buy sto...,2021-09-17,17,9,2021,,{},Discussion,1.0,1,1.0,1,1.0,1.0
--------sam,2,She Definitely Bought TSLA Puts 🐻 Holy shit NB...,,2020-10-02 2020-03-12,2 12,10 3,2020 2020,,{},Meme Discussion,0.0,0,1.0,2,0.5,1.0
------13------,1,Robinhood allowing limited shares to be bought...,,2021-01-28,28,1,2021,,{},Discussion,0.0,0,1.0,1,1.0,1.0
-----1------2,1,Is it worth investing more now?,"Idk how to word it better, im pretty retarded....",2021-01-25,25,1,2021,,{},Discussion,0.0,0,1.0,1,1.0,1.0


## Count Flairs

In [102]:
df_author_posts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 396126 entries, ------------------f to zzzzzzzzzzzzzzzzs
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 396126 non-null  int64  
 1   title              396126 non-null  object 
 2   selftext           396126 non-null  object 
 3   date               396126 non-null  object 
 4   day                396126 non-null  object 
 5   month              396126 non-null  object 
 6   year               396126 non-null  object 
 7   tickers_           396126 non-null  object 
 8   tickers_count_     396126 non-null  object 
 9   link_flair_text    396126 non-null  object 
 10  comments_mean      396126 non-null  float64
 11  comments_sum       396126 non-null  int64  
 12  score_mean         396126 non-null  float64
 13  score_sum          396126 non-null  int64  
 14  upvote_ratio_mean  396126 non-null  float64
 15  upvote_ratio_sum   396126 n

In [110]:
df_author_posts['flairs'] = df_author_posts['link_flair_text'].apply(lambda x: list(x.split(" ")))


In [112]:
df_author_posts["flairs_count"] = df_author_posts['flairs'].apply(lambda x: dict(Counter(list(x))))

In [113]:
df_author_posts.head(3)

,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count
author,,,,,,,,,,,,,,,,,,
------------------f,1,All my homies hate options. $GEVO 🚀🚀,,2020-12-03,3,12,2020,['GEVO'],{'GEVO': 1},Stocks,0.0,0,1.0,1,1.0,1.0,[Stocks],{'Stocks': 1}
--------Username----,1,Is berkshire B spiking soon?,Im 17 and I gave $700 to my brother to buy sto...,2021-09-17,17,9,2021,,{},Discussion,1.0,1,1.0,1,1.0,1.0,[Discussion],{'Discussion': 1}
--------sam,2,She Definitely Bought TSLA Puts 🐻 Holy shit NB...,,2020-10-02 2020-03-12,2 12,10 3,2020 2020,,{},Meme Discussion,0.0,0,1.0,2,0.5,1.0,"[Meme, Discussion]","{'Meme': 1, 'Discussion': 1}"


# Sentiment by Authors

In [96]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacytextblob.spacytextblob import SpacyTextBlob
import json

In [ ]:
pip install SpacyTextBlob

In [97]:
# nlp = spacy.load("en_core_web_lg")
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.blank("en")
if "spacytextblob" not in nlp.pipe_names:
    nlp.add_pipe("spacytextblob")
    nlp.rename_pipe("spacytextblob", "sentiment")
print(nlp.pipe_names)

['sentiment']


In [116]:

def get_sentiment(title, body):
    if(len(body) < 3689602):
      nlp_title = nlp(title)
      nlp_body = nlp(str(body))
      title_tokens = [token.text for token in nlp_title]
      body_tokens = [token.text for token in nlp_body]
      #ngrams = ngrams.append(list(textacy.extract.basics.ngrams(nlp_title, 3)))
      #print(pd.Series([nlp_title._.polarity, nlp_title._.subjectivity,
      #                  nlp_body._.polarity, nlp_body._.subjectivity,title_tokens,body_tokens]))
      return pd.Series([nlp_title._.polarity, nlp_title._.subjectivity,
                        nlp_body._.polarity, nlp_body._.subjectivity,title_tokens,body_tokens])
    else:
      return pd.Series([0, 0,
                       0, 0,0,0])



In [ ]:
nlp.max_length = 3689602  
df_author_posts[["title_polarity", "title_subjectivity", "body_polarity", "body_subjectivity","title_tokens","selftext_tokens"]] = df_author_posts.apply(lambda row: get_sentiment(row["title"], row["selftext"]), axis=1)


In [120]:
df_author_posts["polarity"] = (df_author_posts["title_polarity"] + df_author_posts["body_polarity"])

In [121]:

df_author_posts.head()

,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity
author,,,,,,,,,,,,,,,,,,,,,,,,,
------------------f,1,All my homies hate options. $GEVO 🚀🚀,,2020-12-03,3,12,2020,['GEVO'],{'GEVO': 1},Stocks,0.0,0,1.0,1,1.0,1.0,[Stocks],{'Stocks': 1},-0.800000,0.900000,0.000000,0.000000,"[All, my, homies, hate, options, ., $, GEVO, 🚀...",[ ],-0.800000
--------Username----,1,Is berkshire B spiking soon?,Im 17 and I gave $700 to my brother to buy sto...,2021-09-17,17,9,2021,,{},Discussion,1.0,1,1.0,1,1.0,1.0,[Discussion],{'Discussion': 1},0.000000,0.000000,0.700000,0.600000,"[Is, berkshire, B, spiking, soon, ?]","[I, m, 17, and, I, gave, $, 700, to, my, broth...",0.700000
--------sam,2,She Definitely Bought TSLA Puts 🐻 Holy shit NB...,,2020-10-02 2020-03-12,2 12,10 3,2020 2020,,{},Meme Discussion,0.0,0,1.0,2,0.5,1.0,"[Meme, Discussion]","{'Meme': 1, 'Discussion': 1}",-0.100000,0.650000,0.000000,0.000000,"[She, Definitely, Bought, TSLA, Puts, 🐻, Holy,...",[ ],-0.100000
------13------,1,Robinhood allowing limited shares to be bought...,,2021-01-28,28,1,2021,,{},Discussion,0.0,0,1.0,1,1.0,1.0,[Discussion],{'Discussion': 1},-0.071429,0.142857,0.000000,0.000000,"[Robinhood, allowing, limited, shares, to, be,...",[ ],-0.071429
-----1------2,1,Is it worth investing more now?,"Idk how to word it better, im pretty retarded....",2021-01-25,25,1,2021,,{},Discussion,0.0,0,1.0,1,1.0,1.0,[Discussion],{'Discussion': 1},0.400000,0.300000,0.180519,0.569156,"[Is, it, worth, investing, more, now, ?]","[Idk, how, to, word, it, better, ,, i, m, pret...",0.580519


In [122]:
df_author_posts.to_csv('/content/sample_data/WSB_users_featurs.csv',sep=';')

In [123]:
df_top_users =df_author_posts[df_author_posts['id']> 10]

In [125]:
df_top_users.to_csv('/content/drive/MyDrive/WSB/WSB_top_users_features.csv',sep=';')